In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/My Drive/privacy')

In [ ]:
!pip install -e .

In [ ]:
tensorflow_version 1.x

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from time import strftime, localtime

import os
from absl import app
from tensorflow.python.platform import flags

from distutils.version import LooseVersion

import numpy as np
import tensorflow as tf

from privacy.analysis import privacy_ledger
from privacy.analysis.rdp_accountant import compute_rdp_from_ledger
from privacy.analysis.rdp_accountant import get_privacy_spent
from privacy.optimizers import dp_optimizer

if LooseVersion(tf.__version__) < LooseVersion('2.0.0'):
    GradientDescentOptimizer = tf.train.GradientDescentOptimizer
    AdamOptimizer = tf.train.AdamOptimizer
    AdagradOptimizer = tf.train.AdagradOptimizer
    MomentumOptimizer = tf.train.MomentumOptimizer
else:
    GradientDescentOptimizer = tf.optimizers.SGD  # pylint: disable=invalid-name

In [ ]:
model_base_path = '/home/fanghb/dp-sgd/fork/privacy/'

In [ ]:
# TODO 模型和tensorboard存放的位置
filewriter_path = model_base_path + 'dpsgd' + "_" + 'momentum' + "/tensorboard"
checkpoint_path = model_base_path + 'dpsgd' + "_" + 'momentum' + "/checkpoints"

# Create parent path if it doesn't exist
if not os.path.isdir(checkpoint_path):
    os.makedirs(checkpoint_path)
if not os.path.isdir(filewriter_path):
    os.makedirs(filewriter_path)

In [ ]:
X = tf.placeholder(tf.float32, [600, 28, 28])
Y = tf.placeholder(tf.int64, [600])

def cnn_model_fn(features, labels, mode):
    """Model function for a CNN."""

    # Define CNN architecture using tf.keras.layers.
    
    input_layer = tf.reshape(features, [-1, 28, 28, 1])
    

    y = tf.keras.layers.Conv2D(16, 8,
                               strides=2,
                               padding='same',
                               activation='relu').apply(input_layer)
    y = tf.keras.layers.MaxPool2D(2, 1).apply(y)
    y = tf.keras.layers.Conv2D(32, 4,
                               strides=2,
                               padding='valid',
                               activation='relu').apply(y)
    y = tf.keras.layers.MaxPool2D(2, 1).apply(y)
    y = tf.keras.layers.Flatten().apply(y)
    y = tf.keras.layers.Dense(32, activation='relu').apply(y)
    logits = tf.keras.layers.Dense(10).apply(y)

    # Calculate accuracy.
    correct_pred = tf.equal(tf.argmax(logits, 1), labels)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Calculate loss as a vector (to support microbatches in DP-SGD).
    vector_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=labels, logits=logits)
    # Define mean of loss across minibatch (for reporting through tf.Estimator).
    scalar_loss = tf.reduce_mean(vector_loss)

    
    ledger = privacy_ledger.PrivacyLedger(
            population_size=60000,
            selection_probability=(600 / 60000))

        # Use DP version of GradientDescentOptimizer. Other optimizers are
        # available in dp_optimizer. Most optimizers inheriting from
        # tf.train.Optimizer should be wrappable in differentially private
        # counterparts by calling dp_optimizer.optimizer_from_args().
        
       
    optimizer = dp_optimizer.DPAdamGaussianOptimizer(
                l2_norm_clip=4.0,
                noise_multiplier=4.0,
                num_microbatches=300,
                ledger=ledger,
                learning_rate=0.001,
                )

        
    opt_loss = vector_loss
    
    
    global_step = tf.train.get_global_step()
    train_op = optimizer.minimize(loss=opt_loss, global_step=global_step)
    # In the following, we pass the mean of the loss (scalar_loss) rather than
    # the vector_loss because tf.estimator requires a scalar loss. This is only
    # used for evaluation and debugging by tf.estimator. The actual loss being
    # minimized is opt_loss defined above and passed to optimizer.minimize().
    return train_op, scalar_loss, accuracy

In [ ]:
def load_Fashionmnist():
    """Loads Fashion-MNIST and preprocesses to combine training and validation data."""
    train, test = tf.keras.datasets.fashion_mnist.load_data()
    train_data, train_labels = train
    test_data, test_labels = test

    train_data = np.array(train_data, dtype=np.float32) / 255
    test_data = np.array(test_data, dtype=np.float32) / 255

    train_labels = np.array(train_labels, dtype=np.int32)
    test_labels = np.array(test_labels, dtype=np.int32)

    print(train_labels.shape, test_data.shape)

    assert train_data.min() == 0.
    assert train_data.max() == 1.
    assert test_data.min() == 0.
    assert test_data.max() == 1.
    assert train_labels.ndim == 1
    assert test_labels.ndim == 1

    return train_data, train_labels, test_data, test_labels

In [ ]:
def generate_next_batch(data, label, batch_size, shffule=False):
    print(type(data), data.shape, label.shape, batch_size, "---", data.shape[0], data.shape[1], data.shape[2])
    if shffule:
        permutation = np.random.permutation(data.shape[0])
        data = data[permutation]
        label = label[permutation]
    size = data.shape[0] // batch_size
    print(batch_size, "()()", size, "()", data.shape[0])
    for i in range(size):
        yield (data[i * batch_size: (i + 1) * batch_size], label[i * batch_size: (i + 1) * batch_size])

In [ ]:
train_op, opt_loss, opt_accuracy = cnn_model_fn(X, Y,None)

In [ ]:
tf.summary.scalar('loss', opt_loss)
tf.summary.scalar('accuracy', opt_accuracy)

In [ ]:
train_data, train_labels, test_data, test_labels = load_Fashionmnist()

In [ ]:
def main(unused_argv):
    merged = tf.summary.merge_all()
    # Initialize the FileWriter
    writer = tf.summary.FileWriter(filewriter_path)

    # Initialize an saver for store model checkpoints
    saver = tf.train.Saver(var_list=tf.trainable_variables())

    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        tf.logging.set_verbosity(tf.logging.INFO)
        

        # Load training and test data.
        print(train_data.shape, train_labels.shape)
        # Training loop.
        for epoch in range(1, 100 + 1):

            print("At epcho %d" % epoch)

            # Train the model for one epoch.
            gen = generate_next_batch(data=train_data, label=train_labels, batch_size=600, shffule=True)
            train_acc = 0.
            train_loss = 0.
            train_count = 0
            for img_batch, label_batch in gen:
                _, loss, acc = sess.run([train_op, opt_loss, opt_accuracy], feed_dict={X: img_batch,
                                                                                       Y: label_batch})
                train_acc += acc
                train_loss += loss
                train_count += 1
            train_acc /= train_count
            train_loss /= train_count
            print('Train accuracy: {}, Train loss : {},  after {} epochs'.format(train_acc, train_loss, epoch))

            gen = generate_next_batch(data=test_data, label=test_labels, batch_size=600, shffule=False)
            test_acc = 0.
            test_loss = 0.
            test_count = 0
            for img_batch, label_batch in gen:
                loss, acc = sess.run([opt_loss, opt_accuracy], feed_dict={X: img_batch,
                                                                          Y: label_batch})
                test_acc += acc
                test_loss += loss
                test_count += 1
            test_acc /= test_count
            test_loss /= test_count
            print('Test accuracy: {}, Test loss : {},  after {} epochs'.format(test_acc, test_loss, epoch))

In [ ]:
tf.app.run(main)